In [26]:
import pandas as pd
import numpy as np
from scipy.stats import rankdata

In [27]:
a = pd.read_csv("../submissions/data_v2_0_80_lgb_val0841866.csv")
b = pd.read_csv("../submissions/data_v3_80_100_lgb_7000it_0837796.csv")
c = pd.read_csv("../submissions/gmean_2logits_1ratiobased.csv")

In [28]:
# geometric mean
#ff.CustomerInterest = (ff.CustomerInterest*ll.CustomerInterest) ** (1/2)

In [29]:
# or arithmetic mean
#ff.CustomerInterest = (4/5)*(ff.CustomerInterest) + (1/5)*(ll.CustomerInterest)

In [30]:
# rank mean
a.CustomerInterest = 5*rankdata(a.CustomerInterest) + rankdata(b.CustomerInterest) + rankdata(c.CustomerInterest)
a.CustomerInterest = (a.CustomerInterest - a.CustomerInterest.min()) / (a.CustomerInterest.max() - a.CustomerInterest.min())

In [31]:
# check rank correlation with the best submission
from scipy.stats import spearmanr
best = pd.read_csv("../submissions/data_v2_0_80_lgb_val0841866.csv")
spearmanr(a["CustomerInterest"], best.CustomerInterest)

SpearmanrResult(correlation=0.9783543519141684, pvalue=0.0)

In [32]:
a.to_csv('../submissions/rankmean_3submits.csv', index=False, float_format = "%.8f")